# Homework 5

## FINM 35700 - Spring 2025

### UChicago Financial Mathematics

### Due Date: 2025-04-29

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on following data files:

Government and corporate bonds
- the bond symbology file `bond_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the bond market data file `bond_market_prices_eod`,

Interest Rate & Credit Default Swaps
- the SOFR OIS symbology file `sofr_swap_symbology`,
- the SOFR swaps market data file `sofr_swaps_market_data_eod`,
- the CDS spreads market data file `cds_market_data_eod`.

In [1]:
# import tools from previous homeworks
from credit_market_tools import *

# Use static calculation/valuation date of 2024-12-13, matching data available in the market prices EOD file
calc_date = ql.Date(13, 12, 2024)
ql.Settings.instance().evaluationDate = calc_date

# Calculation/valuation date as pd datetime
as_of_date = pd.to_datetime('2024-12-13')

-----------------------------------------------------------
# Problem 1: Credit Default Swaps (hazard rate model)

## When computing sensitivities, assume "everything else being equal" (ceteris paribus).

For a better understanding of dependencies, you can use the CDS valuation formulas in the simple hazard rate model (formulas[45] and [46] in Lecture 4).

\begin{align}
PV_{CDS\_PL}\left(c,r,h,R,T\right) = \frac{c}{4 \cdot \left(e^{\left(r+h\right)/4}-1 \right)} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right] \simeq \frac{c}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
PV_{CDS\_DL}\left(c,r,h,R,T\right) = \frac{\left(1-R\right)\cdot h}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
PV_{CDS} = PV_{CDS\_PL} - PV_{CDS\_DL} \simeq \frac{c - \left(1-R\right)\cdot h}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
CDS\_ParSpread = c \cdot \frac{PV_{CDS\_DL}}{PV_{CDS\_PL}} \simeq \left(1-R\right)\cdot h
\end{align}


## a. True or False (CDS Premium Leg PV)

1. CDS premium leg PV is increasing in CDS Par Spread
    - False, the opposite is the case.
2. CDS premium leg PV is increasing in interest rate
    - False, the opposite is the case.
2. CDS premium leg PV is increasing in hazard rate
    - False, the opposite is the case.
3. CDS premium leg PV is increasing in recovery rate
    - False, premium leg is independent of recovery rate.
4. CDS premium leg PV is increasing in coupon
    - True, premium leg PV is linear in coupon.
5. CDS premium leg PV is increasing in CDS maturity
    - True, more coupons are being added by extending the maturity.


## b. True or False (CDS Default Leg PV)

1. CDS default leg PV is increasing in CDS Par Spread
    - True, since the credit default risk increases.
2. CDS default leg PV is increasing in interest rate
    - False, the opposite is the case, since the risk-free discount factors decrease.
3. CDS default leg PV is increasing in hazard rate
    - True, since the credit default risk increases.
4. CDS default leg PV is increasing in recovery rate
    - False, the opposite is the case, since the expecte loss decreases.
5. CDS default leg PV is increasing in coupon
    - False, default leg is independent of the coupon.
6. CDS default leg PV is increasing in CDS maturity
    - True, since additional default risk is being added by extending the maturity.

## c. True or False (CDS PV)

1. CDS PV is increasing in CDS Par Spread
    - False, the opposite is the case, since the credit default risk increases.
2. CDS PV is increasing in interest rate
    - False, the opposite is the case, since the credit default risk increases.
3. CDS PV is increasing in hazard rate
    - False, the opposite is the case, since the credit default risk increases.
4. CDS PV is increasing in recovery rate
    - True, the opposite is the case, since the expected recovery increases.
5. CDS PV is increasing in coupon
    - True, since the premium leg PV increases
6. CDS PV is increasing in CDS maturity
    - False, this is only the case if CDS coupon > CDS Par Spread.

## d. True or False (CDS Par Spread)


1. CDS Par Spread is increasing in interest rates
    - False, no general statement can be made.
2. CDS Par Spread is increasing in hazard rate
    - True, CDS_Par_Spread ~= Hazard_Rate x (1 - Recovery)
3. CDS Par Spread is increasing in recovery rate
    - False, the opposite is the case
4. CDS Par Spread is increasing in coupon
    - Flase, CDS Par Spread is independent of the coupon
5. CDS Par Spread is increasing in CDS maturity
    - False, for distressed issuers the opposite is the case (credit risk is concentraded in the fron end of the curve)

-----------------------------------------------------------
# Problem 2: Perpetual CDS
We are interested in a perpetual CDS contract (infinite maturity) on a face notional of $100, flat interest rate of 4% and coupon of 1% (quarterly payments).

For simplicity, we assuming a flat hazard rate of 2% per annum, a recovery rate of 40%, T+0 settlement and zero accrued.

Use the simple CDS valuation formulas derived in Session 4 as a template.

The value of the perpetual CDS is obtained as a limit case of the simple CDS valuation formulas derived in Session 4, for $T \to \infty$.

\begin{align}
PV_{CDS\_PL}\left(c,r,h,R, \infty \right) = \frac{c}{4 \cdot \left(e^{\left(r+h\right)/4}-1 \right)} \simeq \frac{c}{r+h}
\end{align}

\begin{align}
PV_{CDS\_DL}\left(c,r,h,R,\infty \right) = \frac{\left(1-R\right)\cdot h}{r+h} 
\end{align}

\begin{align}
PV_{CDS} = PV_{CDS\_PL} - PV_{CDS\_DL}
\end{align}

\begin{align}
CDS\_ParSpread = c \cdot \frac{PV_{CDS\_DL}}{PV_{CDS\_PL}} \simeq \left(1-R\right)\cdot h
\end{align}


In [2]:
def calc_perpetual_cds_premium_leg_pv(c,r,h,R,face):
    return(c / 4 / (np.exp((r+h)/4)-1) * face)

def calc_perpetual_cds_default_leg_pv(c,r,h,R,face):
    return((1 - R) * h / (r + h) * face)

def calc_perpetual_cds_pv(c,r,h,R,face):
    return(calc_perpetual_cds_premium_leg_pv(c,r,h,R,face) - calc_perpetual_cds_default_leg_pv(c,r,h,R,face))

def calc_perpetual_cds_upfront(c,r,h,R,face):
    return(- calc_perpetual_cds_pv(c,r,h,R,face))

def calc_perpetual_cds_par_spread(c,r,h,R,face):
    return(c * calc_perpetual_cds_default_leg_pv(c,r,h,R,face) / calc_perpetual_cds_premium_leg_pv(c,r,h,R,face))

## a. Compute the fair value of the CDS premium and default legs.


In [3]:
# constant model parameters
r = 0.04
c = 0.01
h = 0.02
R = 0.40
face = 100

# perpetual_cds_premium_leg_pv
perpetual_cds_premium_leg_pv = calc_perpetual_cds_premium_leg_pv(c,r,h,R,face)
print('perpetual_cds_premium_leg_pv:', round(perpetual_cds_premium_leg_pv, 2))

# perpetual_cds_default_leg_pv
perpetual_cds_default_leg_pv = calc_perpetual_cds_default_leg_pv(c,r,h,R,face)
print('perpetual_cds_default_leg_pv:', round(perpetual_cds_default_leg_pv, 2))


perpetual_cds_premium_leg_pv: 16.54
perpetual_cds_default_leg_pv: 20.0


## b. Compute the CDS PV, the CDS Upfront and the CDS Par Spread.

In [4]:
perpetual_cds_pv = calc_perpetual_cds_pv(c,r,h,R,face)
print('perpetual_cds_pv:', round(perpetual_cds_pv, 2))

perpetual_cds_upfront = calc_perpetual_cds_upfront(c,r,h,R,face)
print('perpetual_cds_upfront:', round(perpetual_cds_upfront, 2))

perpetual_cds_par_spread_bps = calc_perpetual_cds_par_spread(c,r,h,R,face) * 1e4
print('perpetual_cds_par_spread_bps:', round(perpetual_cds_par_spread_bps, 2))

perpetual_cds_par_spread_approx_bps = (1 - R) * h * 1e4
print('perpetual_cds_par_spread_approx_bps:', round(perpetual_cds_par_spread_approx_bps, 2))


perpetual_cds_pv: -3.46
perpetual_cds_upfront: 3.46
perpetual_cds_par_spread_bps: 120.9
perpetual_cds_par_spread_approx_bps: 120.0


## c. Compute the following CDS risk sensitivities:
- IR01 (PV sensitivity to Interest Rate change of '-1bp')
- HR01 (PV sensitivity to Hazard Rate change of '-1bp')
- REC01 (PV sensitivity to Recovery Rate change of '+1%')

using the scenario method.


In [5]:
# CDS IR01 
r_1bp_down = r - 0.0001
perpetual_cds_pv_r_1bp_down = calc_perpetual_cds_pv(c,r_1bp_down,h,R,face)
perpetual_cds_ir01 = perpetual_cds_pv_r_1bp_down - perpetual_cds_pv
print('perpetual_cds_ir01 ($):', round(perpetual_cds_ir01, 2))

# CDS HR01 
h_1bp_down = h - 0.0001
perpetual_cds_pv_h_1bp_down = calc_perpetual_cds_pv(c,r,h_1bp_down,R,face)
perpetual_cds_hr01 = perpetual_cds_pv_h_1bp_down - perpetual_cds_pv
print('perpetual_cds_hr01 ($):', round(perpetual_cds_hr01, 2))

# CDS CS01
perpetual_cds_par_spread_h_1bp_down = calc_perpetual_cds_par_spread(c,r,h_1bp_down,R,face) * 1e4
perpetual_cds_par_spread_bps_delta = perpetual_cds_par_spread_bps - perpetual_cds_par_spread_h_1bp_down
perpetual_cds_cs01 = perpetual_cds_hr01 / perpetual_cds_par_spread_bps_delta
print('perpetual_cds_cs01 ($):', round(perpetual_cds_cs01, 2))

# CDS REC01 
R_1pct_up = R + 0.01
perpetual_cds_pv_R_1pct_up = calc_perpetual_cds_pv(c,r,h,R_1pct_up,face)
perpetual_cds_rec01 = perpetual_cds_pv_R_1pct_up - perpetual_cds_pv
print('perpetual_cds_rec01 ($):', round(perpetual_cds_rec01, 2))



perpetual_cds_ir01 ($): -0.01
perpetual_cds_hr01 ($): 0.09
perpetual_cds_cs01 ($): 0.16
perpetual_cds_rec01 ($): 0.33


## d. At what time T does the (implied) default probability over next 10 years (from $[T, T+10]$) drop to 10%?

\begin{align}
\mathbb{P} \left(\tau \in [T, T+10] \right) = 10/100
\end{align}


\begin{align}
\mathbb{P} \left(\tau \in [T, T+10] \right)
\end{align}

\begin{align}
= \mathbb{P} \left(\tau >T \right) - \mathbb{P} \left(\tau > T+10 \right)
\end{align}

\begin{align}
= e^{-h \cdot T} - e^{-h \cdot \left( T + 10 \right)}
\end{align}

\begin{align}
= e^{-h \cdot T}\cdot \left(1 - e^{-h \cdot 10} \right).
\end{align}


In [6]:
# Calc T:
T = np.log((1-np.exp(-h*10))/(10/100))/h
print('T =', T)

T = 29.7406646011763


-----------------------------------------------------------
# Problem 3: Pricing risky bonds in the hazard rate model
## This is building upon
- Homework 2 "Problem 2: US Treasury yield curve calibration (On-The-Runs)",
- Homework 4 "Problem 2: US SOFR swap curve calibration" and
- Homework 4 "Problem 3: CDS Hazard Rate calibration".

## a. Prepare the market data
### Load the symbology + market data dataframes. Calibrate the following curves as of 2024-12-13:
- the "on-the-run" US Treasury curve,
- the US SOFR curve and 
- the IBM CDS hazard rate curve (on the top of SOFR discount curve).


In [7]:
# US Treasury "On-The-Run" yield curve calibration
# Follow Homework 2 Problem 2 and populate the US Treasury On-The-Run symbology + market data frame

# Load bond_symbology.xlsx
bond_symbology  = pd.read_excel('./data/bond_symbology.xlsx')
bond_symbology  = bond_symbology[bond_symbology['cpn_type'] == 'FIXED']

# Add term and TTM columns
bond_symbology['term'] = (bond_symbology['maturity'] - bond_symbology['start_date']).dt.days / 365.25
bond_symbology['TTM'] = (bond_symbology['maturity'] - as_of_date).dt.days / 365.25

# Load bond_market_prices_eod
bond_market_prices_eod = pd.read_excel('./data/bond_market_prices_eod.xlsx')

# Add mid prices and yields
bond_market_prices_eod['midPrice'] = 0.5*(bond_market_prices_eod['bidPrice'] + bond_market_prices_eod['askPrice'])
bond_market_prices_eod['midYield'] = 0.5*(bond_market_prices_eod['bidYield'] + bond_market_prices_eod['askYield'])

# Load govt_on_the_run, as of 2024-12-13
govt_on_the_run = pd.read_excel('./data/govt_on_the_run.xlsx')

# Keep OTR treasuries only
govt_on_the_run_simple = govt_on_the_run[~govt_on_the_run['ticker'].str.contains('B|C')]

# Create symbology for on-the-run treasuries only
govt_symbology_otr = bond_symbology[bond_symbology['isin'].isin(govt_on_the_run_simple['isin'])].copy()
govt_symbology_otr = govt_symbology_otr.sort_values(by='maturity')

# Merge market data as of 2024-12-13 into treasury OTR symbology
govt_combined_otr = govt_symbology_otr.merge(bond_market_prices_eod,  on=['class','ticker','figi','isin'])
display(govt_combined_otr.head())

# tsy_yield_curve calibration
tsy_yield_curve = calibrate_yield_curve_from_frame(calc_date, govt_combined_otr, 'midPrice')
tsy_yield_curve_handle = ql.YieldTermStructureHandle(tsy_yield_curve)




,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,term,TTM,date,bidPrice,askPrice,accrued,bidYield,askYield,midPrice,midYield
0,T,Govt,BBG01QZFYJV6,US91282CLY56,US91282CLY56,T 4 1/4 11/30/26,US TREASURY N/B,US GOVERNMENT,4.250,FIXED,...,1.993155,1.963039,2024-12-13,100.0000,100.0078,0.1875,4.249,4.245,100.0039,4.2470
1,T,Govt,BBG01R4Z7Y32,US91282CMB45,US91282CMB45,T 4 12/15/27,US TREASURY N/B,US GOVERNMENT,4.000,FIXED,...,2.995209,3.003422,2024-12-13,99.3750,99.3828,0.0117,4.224,4.221,99.3789,4.2225
2,T,Govt,BBG01QZFYD58,US91282CMA61,US91282CMA61,T 4 1/8 11/30/29,US TREASURY N/B,US GOVERNMENT,4.125,FIXED,...,4.993840,4.963723,2024-12-13,99.4375,99.4453,0.1816,4.252,4.250,99.4414,4.2510
3,T,Govt,BBG01QZFYCF9,US91282CLZ22,US91282CLZ22,T 4 1/8 11/30/31,US TREASURY N/B,US GOVERNMENT,4.125,FIXED,...,6.992471,6.962355,2024-12-13,98.7969,98.8125,0.1816,4.327,4.324,98.8047,4.3255
4,T,Govt,BBG01QKHSMP5,US91282CLW90,US91282CLW90,T 4 1/4 11/15/34,US TREASURY N/B,US GOVERNMENT,4.250,FIXED,...,9.998631,9.921971,2024-12-13,98.8125,98.8281,0.3633,4.399,4.397,98.8203,4.3980


In [8]:
# SOFR risk-free yield curve calibration
# Follow Homework 4 Problem 2 and populate the SOFR symbology + market data frame

# sofr_symbology
sofr_symbology = pd.read_excel('./data/sofr_swaps_symbology.xlsx')
sofr_symbology.set_index('figi',inplace=True)
display(sofr_symbology)

# sofr_market_quotes
sofr_market_quotes = pd.read_excel('./data/sofr_swaps_market_data_eod.xlsx')

# sofr_combined
sofr_combined = sofr_symbology.merge(sofr_market_quotes[sofr_market_quotes['date'] == as_of_date], how='left', on=['figi'])
display(sofr_combined.head())

# sofr_yield_curve calibration
sofr_yield_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_combined, 'midRate')
sofr_yield_curve_handle = ql.YieldTermStructureHandle(sofr_yield_curve)


,ticker,class,bbg,name,tenor,type,dcc,exchange,country,currency,status
figi,,,,,,,,,,,
BBG00KFWPJJ9,USOSFR1,Curncy,USOSFR1 Curncy,USD OIS ANN VS SOFR 1Y,1,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPJX3,USOSFR2,Curncy,USOSFR2 Curncy,USD OIS ANN VS SOFR 2Y,2,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPK15,USOSFR3,Curncy,USOSFR3 Curncy,USD OIS ANN VS SOFR 3Y,3,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPK51,USOSFR5,Curncy,USOSFR5 Curncy,USD OIS ANN VS SOFR 5Y,5,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPK79,USOSFR7,Curncy,USOSFR7 Curncy,USD OIS ANN VS SOFR 7Y,7,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPKB4,USOSFR10,Curncy,USOSFR10 Curncy,USD OIS ANN VS SOFR 10Y,10,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPKF0,USOSFR20,Curncy,USOSFR20 Curncy,USD OIS ANN VS SOFR 20Y,20,SWAP,ACT/360,NONE,US,USD,ACTV
BBG00KFWPKH8,USOSFR30,Curncy,USOSFR30 Curncy,USD OIS ANN VS SOFR 30Y,30,SWAP,ACT/360,NONE,US,USD,ACTV


,figi,ticker,class,bbg,name,tenor,type,dcc,exchange,country,currency,status,date,bidRate,askRate,midRate
0,BBG00KFWPJJ9,USOSFR1,Curncy,USOSFR1 Curncy,USD OIS ANN VS SOFR 1Y,1,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,4.1858,4.1958,4.19080
1,BBG00KFWPJX3,USOSFR2,Curncy,USOSFR2 Curncy,USD OIS ANN VS SOFR 2Y,2,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,4.0524,4.0585,4.05545
2,BBG00KFWPK15,USOSFR3,Curncy,USOSFR3 Curncy,USD OIS ANN VS SOFR 3Y,3,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,3.9883,3.9944,3.99135
3,BBG00KFWPK51,USOSFR5,Curncy,USOSFR5 Curncy,USD OIS ANN VS SOFR 5Y,5,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,3.9133,3.9181,3.91570
4,BBG00KFWPK79,USOSFR7,Curncy,USOSFR7 Curncy,USD OIS ANN VS SOFR 7Y,7,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,3.8937,3.8991,3.89640


In [9]:
# Credit curve calibration using IBM CDS par spreads 
# Follow Homework 4 Problem 3 and create the IBM hazard rate curve

# cds_market_quotes
cds_market_quotes = pd.read_excel('./data/cds_market_data_eod.xlsx')

# Create par spreads (bps) dataframe
par_spread_col_names = [f'par_spread_{n}y' for n in [1,2,3,5,7,10]]
cds_par_spreads_df = cds_market_quotes.set_index('date')[par_spread_col_names]

cds_par_spreads = list(cds_par_spreads_df.loc[as_of_date])
print(cds_par_spreads)

# cds_recovery_rate
cds_recovery_rate = 0.4

# hazard_rate_curve
hazard_rate_curve = calibrate_cds_hazard_rate_curve(calc_date, sofr_yield_curve_handle, cds_par_spreads, cds_recovery_rate)

# hazard_rate_curve calibrated to IBM CDS par spreads
default_prob_curve_handle = ql.DefaultProbabilityTermStructureHandle(hazard_rate_curve)

[10.9082, 15.6009, 22.4095, 35.4733, 50.8816, 61.462]


## b. Create the IBM risky bond objects
### Identify the following 3 IBM fixed rate bonds in the symbology table and create the corresponding fixed rate bonds (3 bond objects).

- security = 'IBM 3.3 01/27/27' / figi = 'BBG00FVNGFP3'
- security = 'IBM 6 1/2 01/15/28' / figi = 'BBG000058NM4'
- security = 'IBM 3 1/2 05/15/29' / figi = 'BBG00P3BLH14'


Use the create_bond_from_symbology() function (discussed in from Homework 2) to create the bonds objects.

Display the bond cashflows using the get_bond_cashflows() function.

In [10]:
# corp_symbology_ibm
corp_symbology_ibm = bond_symbology[(bond_symbology.ticker == 'IBM') & (bond_symbology.cpn_type == 'FIXED')]

# corp_combined_ibm
corp_combined_ibm = corp_symbology_ibm.merge(bond_market_prices_eod, how='inner', on=['class', 'ticker', 'isin', 'figi'])
corp_combined_ibm.set_index('figi',inplace=True)

# Keep selected IBM bonds only
ibm_selected_figis = ['BBG00FVNGFP3', 'BBG000058NM4', 'BBG00P3BLH14']

ibm_df = corp_combined_ibm.loc[ibm_selected_figis]

display(ibm_df.T)

figi,BBG00FVNGFP3,BBG000058NM4,BBG00P3BLH14
ticker,IBM,IBM,IBM
class,Corp,Corp,Corp
isin,US459200JR30,US459200AS04,US459200KA85
und_bench_isin,US91282CMB45,US91282CMA61,US91282CMA61
security,IBM 3.3 01/27/27,IBM 6 1/2 01/15/28,IBM 3 1/2 05/15/29
name,IBM CORP,IBM CORP,IBM CORP
type,GLOBAL,US DOMESTIC,GLOBAL
coupon,3.3,6.5,3.5
cpn_type,FIXED,FIXED,FIXED
dcc,30/360,30/360,30/360


In [11]:
# Create ibm_bond_objects
ibm_bond_objects = [ create_bond_from_symbology(df_row.to_dict()) for index, df_row in ibm_df.iterrows()]

# List the bond cashflows
for i in range(0, 3):
    print('Bond cashflows for', ibm_df.iloc[i]['security'])
    display(get_bond_cashflows(ibm_bond_objects[i], calc_date)) 


Bond cashflows for IBM 3.3 01/27/27


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
15,"January 27th, 2025",0.122222,1.65
16,"July 27th, 2025",0.622222,1.65
17,"January 27th, 2026",1.122222,1.65
18,"July 27th, 2026",1.622222,1.65
19,"January 27th, 2027",2.122222,1.65
20,"January 27th, 2027",2.122222,100.00


Bond cashflows for IBM 6 1/2 01/15/28


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
54,"January 15th, 2025",0.088889,3.25
55,"July 15th, 2025",0.588889,3.25
56,"January 15th, 2026",1.088889,3.25
57,"July 15th, 2026",1.588889,3.25
58,"January 15th, 2027",2.088889,3.25
59,"July 15th, 2027",2.588889,3.25
60,"January 15th, 2028",3.088889,3.25
61,"January 15th, 2028",3.088889,100.00


Bond cashflows for IBM 3 1/2 05/15/29


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
11,"May 15th, 2025",0.422222,1.75
12,"November 15th, 2025",0.922222,1.75
13,"May 15th, 2026",1.422222,1.75
14,"November 15th, 2026",1.922222,1.75
15,"May 15th, 2027",2.422222,1.75
16,"November 15th, 2027",2.922222,1.75
17,"May 15th, 2028",3.422222,1.75
18,"November 15th, 2028",3.922222,1.75
19,"May 15th, 2029",4.422222,1.75
20,"May 15th, 2029",4.422222,100.00


## c. Compute CDS-implied (intrinsic) prices for the IBM fixd rate bonds

Price the 3 IBM bonds using the CDS-calibrated hazard rate curve for IBM (via RiskyBondEngine, discussed in the QuantLib Advanced examples notebook).

Display the clean prices and yields for the 3 test bonds. 

In [12]:
# flat_recovery_rate
flat_recovery_rate = 0.40

# Risky bond engine uses the calibrated CDS hazard rate curve for pricing credit default risk 
risky_bond_engine = ql.RiskyBondEngine(default_prob_curve_handle, flat_recovery_rate, tsy_yield_curve_handle)

# Model/intrinsic prices and yields
ibm_model_prices = []
ibm_model_yields = []

# Print the clean prices and yields for the 3 test bonds
for i in range(0, 3):
    fixed_rate_bond = ibm_bond_objects[i]
    fixed_rate_bond.setPricingEngine(risky_bond_engine)
    
    corpBondModelPrice = round(fixed_rate_bond.cleanPrice(), 3)
    corpBondModelYield = round(fixed_rate_bond.bondYield(corpBondModelPrice, ql.Thirty360(ql.Thirty360.USA), ql.Compounded, ql.Semiannual) * 100, 3)

    ibm_model_prices.append(corpBondModelPrice)
    ibm_model_yields.append(corpBondModelYield)
    
    
# Display relevant metrics
ibm_df['modelPrice'] = ibm_model_prices
ibm_df['modelYield'] = ibm_model_yields

display(ibm_df[['security', 'modelPrice', 'modelYield']])


,security,modelPrice,modelYield
figi,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,98.065,4.265
BBG000058NM4,IBM 6 1/2 01/15/28,105.793,4.465
BBG00P3BLH14,IBM 3 1/2 05/15/29,95.849,4.548


## d. Compute the "intrinsic" vs market price basis for the IBM bonds

Load the market mid prices and yields from the bond market data dataframe as of 2024-04-19. 

Compute and display the basis between the "CDS-implied intrinsic" vs market prices and yields:

- basisPrice = modelPrice - midPrice
- basisYield = modelYield - midYield


Are the CDS intrinsic prices lower or higher than the bond prices observed on the market? What factors could explain the basis?

In [13]:
# Compute basis
ibm_df['basisPrice'] = ibm_df['modelPrice'] - ibm_df['midPrice']
ibm_df['basisYield'] = ibm_df['modelYield'] - ibm_df['midYield']

# Display relevant metrics
display(ibm_df[['security', 'midPrice', 'modelPrice', 'basisPrice', 'midYield', 'modelYield', 'basisYield']])

,security,midPrice,modelPrice,basisPrice,midYield,modelYield,basisYield
figi,,,,,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,97.4820,98.065,0.5830,4.5615,4.265,-0.2965
BBG000058NM4,IBM 6 1/2 01/15/28,105.3015,105.793,0.4915,4.6315,4.465,-0.1665
BBG00P3BLH14,IBM 3 1/2 05/15/29,95.4065,95.849,0.4425,4.6630,4.548,-0.1150


CDS-implied, intrinsic bond prices are higher than bond market prices.

Following factors could explain the basis dislocation for the 3 IBM bonds:
1. Hazard Rate curve mismatch: the synthetic CDS credit market is underestimating the credit risk in the IBM issuer curve, relative to the cash corporate bond market. This opens the opportunity for Bond vs CDS basis arbitrage trades, as discussed in Session 2. 
2. Risk-free yield curve mismatch: the (synthetic) SOFR yield curve is tighter than the (cash) US Treasury curve. This is usually due to a funding differential for cash vs. synthetic products.
3. Temporarily dislocation: Individual bonds are temporarily dislocated from their "fair value" from the issuer curve (e.g. in a Nelson-Siegel type parametric model). This can happen due to buying vs. selling imbalance in that particular bond.
4. Liquidity discounts: in general, less liquid (e.g. off-the-run) bonds trade at a price discount to more liquid (e.g. on-the-run) bonds. This usually causes a liquidty-implied "richer" basis (wider in yield space).

-----------------------------------------------------------
# Problem 4: Compute scenario sensitivities for risky bonds
## a. Compute scenario IR01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 3. 

Compute the scenario IR01 and Durations using a '-1bp' interest rate shock, as described in Section 6. "Market Data Scenarios" in the QuantLib Basics notebook.

Display the computed scenario IR01 and Durations.

Remember that IR01 = Dirty_Price * Duration.


In [14]:
# Bump interest rate by -1bps (parallel shift)
interest_rate_scenario_1bp_down = ql.SimpleQuote(-0.0001)
tsy_yield_curve_handle_1bp_down = ql.YieldTermStructureHandle(ql.ZeroSpreadedTermStructure(tsy_yield_curve_handle, ql.QuoteHandle(interest_rate_scenario_1bp_down)))
risky_bond_engine_1bp_down = ql.RiskyBondEngine(default_prob_curve_handle, flat_recovery_rate, tsy_yield_curve_handle_1bp_down)

In [15]:
# Model scenario metrics
ibm_scen_prices_1bp_down = []
ibm_scen_IR01 = []
ibm_scen_duration = []
price_1bp_down_list = []
price_base_list = []

# Calculate IR01 and duration
for i in range(0, 3):
    fixed_rate_bond = ibm_bond_objects[i]
    
    # Calc model dirty price for base case
    fixed_rate_bond.setPricingEngine(risky_bond_engine)    
    dirty_price_base = fixed_rate_bond.dirtyPrice()
    
    # Scenario: 1bp down
    fixed_rate_bond.setPricingEngine(risky_bond_engine_1bp_down)    
    price_1bp_down = fixed_rate_bond.cleanPrice()
    ibm_scen_prices_1bp_down.append(price_1bp_down)
    
    # Compute scenario IR01 and duration sensitivities
    price_base = ibm_model_prices[i]
    ir01 = (price_1bp_down - price_base) * 1e4 / 100
    duration = ir01 / dirty_price_base * 100

    price_base_list.append(price_base)
    price_1bp_down_list.append(price_1bp_down)
    ibm_scen_IR01.append(ir01)
    ibm_scen_duration.append(duration)


# Display relevant metrics
ibm_df['price_base'] = price_base_list
ibm_df['price_1bp_down'] = price_1bp_down_list
ibm_df['scen_IR01'] = ibm_scen_IR01
ibm_df['scen_duration'] = ibm_scen_duration

display(ibm_df[['security', 'price_base', 'price_1bp_down', 'scen_IR01', 'scen_duration']])


,security,price_base,price_1bp_down,scen_IR01,scen_duration
figi,,,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,98.065,98.084301,1.930060,1.942909
BBG000058NM4,IBM 6 1/2 01/15/28,105.793,105.822750,2.974988,2.741441
BBG00P3BLH14,IBM 3 1/2 05/15/29,95.849,95.888556,3.955581,4.113942


## b. Compute analytical IR01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 3. 

Compute and display the analytical IR01 and Durations 

Compare the analytic IR01s vs. the scenario IR01s. Are they expected to be similar?

In [16]:
# Analytic metrics
ibm_analytic_durations = []
ibm_analytic_IR01s = []

# Calculate IR01 and duration
for i in range(0, 3):
    fixed_rate_bond = ibm_bond_objects[i]
    
    # Calc model dirty price for base case
    fixed_rate_bond.setPricingEngine(risky_bond_engine)    
    dirty_price_base = fixed_rate_bond.dirtyPrice()
    
    # Compute analytical duration and IR01
    bond_yield_rate = ql.InterestRate(ibm_model_yields[i]/100, ql.Thirty360(ql.Thirty360.USA), ql.Compounded, ql.Semiannual)
    analytic_duration = ql.BondFunctions.duration(fixed_rate_bond, bond_yield_rate)
    analytic_IR01 = analytic_duration * dirty_price_base /100 

    ibm_analytic_durations.append(analytic_duration)
    ibm_analytic_IR01s.append(analytic_IR01)


# Display relevant metrics
ibm_df['analytic_IR01'] = ibm_analytic_IR01s
ibm_df['analytic_duration'] = ibm_analytic_durations

display(ibm_df[['security', 'analytic_IR01', 'analytic_duration']])


,security,analytic_IR01,analytic_duration
figi,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,1.977336,1.990499
BBG000058NM4,IBM 6 1/2 01/15/28,2.949218,2.717695
BBG00P3BLH14,IBM 3 1/2 05/15/29,3.862474,4.017107


In [17]:
# Compare the analytical IR01s vs. the scenario IR01s. Are they expected to be similar?
display(ibm_df[['security', 'scen_IR01', 'analytic_IR01']])

,security,scen_IR01,analytic_IR01
figi,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,1.930060,1.977336
BBG000058NM4,IBM 6 1/2 01/15/28,2.974988,2.949218
BBG00P3BLH14,IBM 3 1/2 05/15/29,3.955581,3.862474


Analytical and scenario IR01s are expected to be similar, since they both represent a -1bp change in the (risk free) interest rate curve.

The difference is caused by  second order (convexity) effects, which are included in the scenario IR01s, but not in the analytical IR01s.

Also, keep in mind that IR01 in the hazard rate model corresponds to DV01 in the flat yield model (since a 1bp change in interest rates causes a 1bp change in the flat bond yield).

## c. Compute scenario CS01s (credit spread sensitivities) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 3. 

Apply a '-1bp' (parallel shift) scenario to the IBM CDS Par Spread quotes and re-calibrate the scenario hazard rate curve. 

Create a new scenario RiskyBondEngine, using the scenario hazard rate curve.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped hazard rate curve) to obtain the '-1bp' scenario CS01 (credit spread sensitivities).

Compare the scenario CS01s vs analytic IR01s. Are they expected to be similar?

In [18]:
# hazard_rate_curve calibration (from IBM CDS par spreads)
cds_par_spreads_1bp_down = [ps - 1 for ps in cds_par_spreads]
print(cds_par_spreads)
print(cds_par_spreads_1bp_down)

# hazard_rate_curve
hazard_rate_curve_1bp_down = calibrate_cds_hazard_rate_curve(calc_date, sofr_yield_curve_handle, cds_par_spreads_1bp_down, cds_recovery_rate)
default_prob_curve_handle_1bp_down = ql.DefaultProbabilityTermStructureHandle(hazard_rate_curve_1bp_down)

# Risky bond engine for CDS Par Spread -1bp scenario
risky_bond_engine_cds_1bp_down = ql.RiskyBondEngine(default_prob_curve_handle_1bp_down, flat_recovery_rate, tsy_yield_curve_handle)


[10.9082, 15.6009, 22.4095, 35.4733, 50.8816, 61.462]
[9.9082, 14.6009, 21.4095, 34.4733, 49.8816, 60.462]


In [19]:
# Calculate CS01
ibm_model_cs01 = []

for i in range(0, 3):
    fixed_rate_bond = ibm_bond_objects[i]     
    price_base = ibm_model_prices[i]   
    
    # set scenario pricing engine
    fixed_rate_bond.setPricingEngine(risky_bond_engine_cds_1bp_down)
    
    # calc credit spread scenario price
    price_cds_1bp_down = fixed_rate_bond.cleanPrice()

    # calc price diff
    price_diff_cds_1bp_down = price_cds_1bp_down - price_base
    
    ibm_model_cs01.append(price_diff_cds_1bp_down * 1e4 / 100)

ibm_df['CS01'] = ibm_model_cs01

# Compare the scenario CS01s vs analytic IR01s. Are they expected to be similar?
display(ibm_df[['security', 'scen_IR01', 'analytic_IR01', 'CS01']])


,security,scen_IR01,analytic_IR01,CS01
figi,,,,
BBG00FVNGFP3,IBM 3.3 01/27/27,1.930060,1.977336,1.987586
BBG000058NM4,IBM 6 1/2 01/15/28,2.974988,2.949218,3.104502
BBG00P3BLH14,IBM 3 1/2 05/15/29,3.955581,3.862474,3.970701


CS01s and IR01s are expected to be similar, since a -1bp change in the credit spread curve causes approximately a -1bp change in the (risky) bond yield curve.

## d. Compute scenario REC01 (recovery rate sensitivity) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 3. 

Apply a +1% scenario bump to the IBM recovery rate (bump the flat_recovery_rate parameter by 1%, from 40% to 41%).

Create a new scenario RiskyBondEngine, using the scenario new recovery rate.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped recovery rate) to obtain the +1% scenario REC01 (recovery rate sensitivity).


In [20]:
# Bump recovery rate by 1% up
flat_recovery_rate_1pct_up = flat_recovery_rate + 0.01
risky_bond_engine_rec_1pct_up = ql.RiskyBondEngine(default_prob_curve_handle, flat_recovery_rate_1pct_up, tsy_yield_curve_handle)

# Calculate REC01
ibm_model_rec01 = []

for i in range(0, 3):
    fixed_rate_bond = ibm_bond_objects[i]
    price_base = ibm_model_prices[i]    
    
    # set scenario pricing engine
    fixed_rate_bond.setPricingEngine(risky_bond_engine_rec_1pct_up)
    
    # calc price diff
    price_rec_1pct_up = fixed_rate_bond.cleanPrice()    
    price_diff_rec_1pct_up = price_rec_1pct_up - price_base
    
    ibm_model_rec01.append(price_diff_rec_1pct_up)

# Display relevant metrics
ibm_df['REC01'] = ibm_model_rec01
print(ibm_df[['security', 'REC01']])


                        security     REC01
figi                                      
BBG00FVNGFP3    IBM 3.3 01/27/27  0.005164
BBG000058NM4  IBM 6 1/2 01/15/28  0.010933
BBG00P3BLH14  IBM 3 1/2 05/15/29  0.022272
